In [1]:

import numpy as np
import pandas as pd 
import numpy as np
import pickle as pkl

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore") 

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [2]:
train_test_data = [train, test]

1.Changing Sex in numerical values

In [3]:
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

2.Fixing the missing values in embarked

In [4]:
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [5]:
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} )

3.Fixing Age

In [6]:
for dataset in train_test_data:
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].median())

train.Age.isnull().sum()

0

4. Adding the titles from names 

In [7]:
for dataset in train_test_data:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.')

In [8]:
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', \
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title']).mean()


title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5}
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

5. Changing the age feature 

In [9]:
#change the age in categorical format
for dataset in train_test_data:
    age_mean = dataset['Age'].mean()
    age_deviation = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()

    random_age = np.random.randint(age_mean - age_deviation, age_mean + age_deviation, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = random_age
    dataset['Age'] = dataset['Age'].astype(int)
train['AgeRange'] = pd.cut(train['Age'], 5)


In [10]:
for dataset in train_test_data:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <=32),'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <=48),'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <=64),'Age'] = 3
    dataset.loc[(dataset['Age'] > 64,'Age')] = 4 


In [11]:
features_drop = ['Name', 'SibSp', 'Parch', 'Ticket', 'Cabin','Fare']
train = train.drop(features_drop, axis=1)
test = test.drop(features_drop, axis=1)
train = train.drop(['PassengerId','AgeRange'], axis=1)

## Prediction

In [12]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']
X_test  = test.drop('PassengerId', axis=1).copy()


In [15]:
train.head()

,Survived,Pclass,Sex,Age,Embarked,Title
0,0,3,0,1,0,1
1,1,1,1,2,1,3
2,1,3,1,1,0,2
3,1,1,1,2,0,3
4,0,3,0,2,0,1


In [14]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,max_depth=5,random_state=1)
clf = clf.fit(X_train, y_train)

pkl.dump(clf, open('model.pkl','wb'))